<a href="https://colab.research.google.com/github/greenrrr/Project-90106-G28/blob/master/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder



def load_data(filename):
    filepath = os.path.join(os.getcwd(), filename)
    data = pd.read_csv('/content/sample_data/cleaned_2.csv', index_col='Id')
    data['Is_External__c'] = data['Is_External__c'].fillna('Internal')

    return data

def remove(dataset):
  dataset = dataset.dropna()
  labels = dataset['StageName']
  train = dataset.drop('StageName',axis = 1)
  for index,col in train.iteritems():
        if str(col[col.notnull()][0]).isdigit() == False:
            train[index] = pd.factorize(train[index])[0].astype(np.uint16)
  return train,labels

def deal_data(raw_data):
    missing_set = {}
    no_missing = {}
    index_missing = {}
    index_no_missing = {}
    labels = {}
    names = []
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

    for index, col in raw_data.iteritems():
        if raw_data[index].isnull().any():
            if len(col[col.isnull()]) / len(col) >= 0.4:  # 如果缺失值大于40% 则用最频繁的值插补
                raw_data[index] = imp.fit_transform(raw_data[index].values.reshape(-1, 1))
                # 剩下六个缺失不大于40%的属性
            else:
                # col.name是属性名，col.values是属性值，col.index是每个属性值对应的Id
                index_missing[index] = col[col.isnull()].index #获取每个属性缺失值的索引
                index_no_missing[index] = col[col.notnull()].index
                names.append(index)

    for index,col in raw_data.iteritems():
        if str(col[col.notnull()][0]).isdigit() == False:
            if index == 'StageName':
                labels_corres = pd.factorize(raw_data[index])[1]
            raw_data[index] = pd.factorize(raw_data[index])[0].astype(np.uint16)

    for i in names:
        labels[i] = raw_data.loc[index_no_missing[i],i]


    raw_data = raw_data.dropna(axis = 1)

    for key,value in index_missing.items():
        missing_set[key] = raw_data.loc[value]
        no_values = index_no_missing[key]
        no_missing[key] = raw_data.loc[no_values]

    return no_missing,missing_set,index_no_missing,index_missing,labels,names


def impute(no_missing,missing_set,index_missing,labels,names,raw_data):
    predict = {}

    for feature in names:
        label = labels[feature]
        train_set = no_missing[feature]
        missing = missing_set[feature]
        predict[feature] = random_forest(train_set,label,missing)

    for key,items in predict.items():
        c = 0
        for i in index_missing[key]:
            raw_data.loc[i,key] = items[c]
            c += 1
    train = raw_data.drop('StageName',axis=1)
    y_label = raw_data['StageName']

    return train,y_label


def random_forest(train,label,missing):
    random_f = RandomForestRegressor()
    random_f.fit(train,label)
    y = random_f.predict(missing)
    return y


def onhot_encoding(dataset):
    enc = OneHotEncoder()
    coding_set = enc.fit_transform(dataset)
    print(coding_set)
    return coding_set

def data_split(train,labels):
    X_train, X_test, y_train, y_test = train_test_split(train, labels, train_size=0.8, random_state=1)

    return X_train, X_test, y_train, y_test

def nn_build():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(300, activation="relu"))
    model.add(tf.keras.layers.Dense(100, activation="relu"))
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    return model


if __name__ == '__main__':
    data = load_data('cleaned_2.csv')
    #train,labels = remove(data)
    no_missing, missing_set, index_no_missing, index_missing, labels, names = deal_data(data)
    X_set,y = impute(no_missing, missing_set, index_missing, labels, names, data)
    X_1, X_test, y_1, y_test = data_split(X_set,y)
    X_train,X_dev,y_train,y_dev = data_split(X_1,y_1)
    model = nn_build()
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=True, validation_data=(X_dev, y_dev), batch_size=10)

    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    print("Testing Accuracy:  {:.4f}".format(accuracy))


Epoch 1/10
388/388 [==============================] - 1s 2ms/step - loss: 7.1823 - accuracy: 0.5599 - val_loss: 2.7675 - val_accuracy: 0.4314
Epoch 2/10
388/388 [==============================] - 1s 2ms/step - loss: 4.2554 - accuracy: 0.5628 - val_loss: 2.5038 - val_accuracy: 0.3653
Epoch 3/10
388/388 [==============================] - 1s 2ms/step - loss: 2.9128 - accuracy: 0.5770 - val_loss: 4.8880 - val_accuracy: 0.6677
Epoch 4/10
388/388 [==============================] - 1s 2ms/step - loss: 3.0756 - accuracy: 0.5870 - val_loss: 4.4196 - val_accuracy: 0.3344
Epoch 5/10
388/388 [==============================] - 1s 2ms/step - loss: 2.0033 - accuracy: 0.5940 - val_loss: 1.3098 - val_accuracy: 0.6481
Epoch 6/10
388/388 [==============================] - 1s 2ms/step - loss: 1.9400 - accuracy: 0.5876 - val_loss: 2.8378 - val_accuracy: 0.6718
Epoch 7/10
388/388 [==============================] - 1s 2ms/step - loss: 2.4038 - accuracy: 0.5824 - val_loss: 1.5827 - val_accuracy: 0.6739
Epoch 